In [ ]:
#create a new geopackage with Geoprocessingtools → "CreateSQLite Database" → Abcd.gpkg

In [ ]:
import arcpy
import re

aprx = arcpy.mp.ArcGISProject("CURRENT")
map_ = aprx.activeMap

gpkg = r"C:\path\to\output.gpkg"  #Abcd.gpkg

used_names = set()

def clean_name(name):
    # Replace invalid characters with underscore
    name = re.sub(r'[^a-zA-Z0-9_]', '_', name)
    # GeoPackage table names must not start with a number
    if name[0].isdigit():
        name = f"layer_{name}"
    # Keep it short (safe limit)
    return name[:60]

def unique_name(name):
    base = name
    i = 1
    while name in used_names:
        name = f"{base}_{i}"
        i += 1
    used_names.add(name)
    return name

for lyr in map_.listLayers():
    # Skip group layers, rasters, tables, broken layers
    if (
        lyr.isFeatureLayer
        and lyr.supports("DATASOURCE")
        and not lyr.isBroken
    ):
        out_name = unique_name(clean_name(lyr.name))
        print(f"Exporting: {lyr.name} → {out_name}")

        arcpy.conversion.FeatureClassToFeatureClass(
            lyr,
            gpkg,
            out_name
        )

print("Done! All vector layers exported to GeoPackage ✅")
